In [4]:
%reload_ext autoreload
%autoreload 2

In [79]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [78]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [58]:
# read one of json file for test purpose"C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized/
data = pd.read_json("C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized/users.json")

In [59]:
# printing 5 rows from the top
data.head()

,id,team_id,name,deleted,color,real_name,tz,tz_label,tz_offset,profile,...,is_owner,is_primary_owner,is_restricted,is_ultra_restricted,is_bot,is_app_user,updated,is_email_confirmed,who_can_share_contact_card,is_invited_user
0,U03T89ACUUW,T03U4J8HMUG,Carlos,False,4bbe2e,Carlos Gross,Africa/Harare,Central Africa Time,7200.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,1.0,0.0,0.0,0.0,False,False,1669395557,0.0,EVERYONE,NaN
1,U03TEPYRM2P,T03U4J8HMUG,Garrett,False,9f69e7,Garrett Bell,Africa/Algiers,Central European Time,3600.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,1.0,1.0,0.0,0.0,False,False,1673852951,1.0,EVERYONE,NaN
2,U03TNP8Q8CT,T03U4J8HMUG,Bethany,False,e7392d,Bethany George,Africa/Algiers,Central European Time,3600.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,False,False,1660570354,0.0,EVERYONE,1.0
3,U03TT5KEYCF,T03U4J8HMUG,Luis,False,e0a729,Luis Ingram,Africa/Algiers,Central European Time,3600.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,False,False,1661268798,1.0,EVERYONE,NaN
4,U03TX2VN6H5,T03U4J8HMUG,Nicole,False,4cc091,Nicole Kim,Africa/Cairo,Eastern European Time,7200.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,False,False,1660904839,1.0,EVERYONE,NaN


In [60]:
# printing 5 items from the bottom
data.tail()

,id,team_id,name,deleted,color,real_name,tz,tz_label,tz_offset,profile,...,is_owner,is_primary_owner,is_restricted,is_ultra_restricted,is_bot,is_app_user,updated,is_email_confirmed,who_can_share_contact_card,is_invited_user
66,U03V8LHPDME,T03U4J8HMUG,Mandy,False,e96699,Mandy Sweeney,America/New_York,Eastern Standard Time,-18000.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,False,False,1667364708,1.0,EVERYONE,NaN
67,U03VAH809FC,T03U4J8HMUG,Paul,False,df3dc0,Paul Brown,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1661515236,0.0,EVERYONE,NaN
68,U0401GU4W6L,T03U4J8HMUG,Kathleen,False,2b6836,Kathleen Phillips,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1661252626,0.0,EVERYONE,NaN
69,U0418T6CD1B,T03U4J8HMUG,Kenneth,False,4cc091,Kenneth Fitzgerald,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1662534866,0.0,EVERYONE,NaN
70,U04718Y7SQ0,T03U4J8HMUG,Audrey,False,9b3b45,Audrey Parsons,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1665649740,0.0,EVERYONE,NaN


In [61]:
# print desccription of the data 
data.describe()

,tz_offset,is_admin,is_owner,is_primary_owner,is_restricted,is_ultra_restricted,updated,is_email_confirmed,is_invited_user
count,57.000000,57.000000,57.000000,57.000000,57.0,57.0,7.100000e+01,57.000000,1.0
mean,3410.526316,0.070175,0.070175,0.017544,0.0,0.0,1.672156e+09,0.859649,1.0
std,12987.489643,0.257713,0.257713,0.132453,0.0,0.0,1.243181e+07,0.350438,NaN
min,-28800.000000,0.000000,0.000000,0.000000,0.0,0.0,1.660570e+09,0.000000,1.0
25%,3600.000000,0.000000,0.000000,0.000000,0.0,0.0,1.663371e+09,1.000000,1.0
50%,10800.000000,0.000000,0.000000,0.000000,0.0,0.0,1.667401e+09,1.000000,1.0
75%,10800.000000,0.000000,0.000000,0.000000,0.0,0.0,1.676618e+09,1.000000,1.0
max,10800.000000,1.000000,1.000000,1.000000,0.0,0.0,1.700820e+09,1.000000,1.0


In [62]:
# know the shape of the data 
data.shape

(71, 21)

In [63]:
# print name of the data coumns
data.columns

Index(['id', 'team_id', 'name', 'deleted', 'color', 'real_name', 'tz',
       'tz_label', 'tz_offset', 'profile', 'is_admin', 'is_owner',
       'is_primary_owner', 'is_restricted', 'is_ultra_restricted', 'is_bot',
       'is_app_user', 'updated', 'is_email_confirmed',
       'who_can_share_contact_card', 'is_invited_user'],
      dtype='object')

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [82]:

#df = pd.read_json(slack_parser("C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized/"))

data_loader = SlackDataLoader("C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized/")

slack_data = data_loader(slack_parser("C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized/"))

ValueError: I/O operation on closed file.

In [ ]:
# which user has the highest number of reply counts?


In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [11]:
# what kind of messages are replied faster than others?

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?